In [1]:
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='Arial Unicode MS')
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
import folium
import json
from glob import glob
import missingno as msno

In [2]:
# Market size data 불러오기
df = pd.read_csv('./datas/market_size_reform.csv', encoding='utf-8')
df_ms = pd.DataFrame(df)
df_ms.head()

,년도,시장규모,산업분류
0,1995,855409035,전체
1,2000,1395749584,전체
2,2005,2058128879,전체
3,2006,2203889676,전체
4,2007,2396507678,전체


In [3]:
df_ms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   년도      352 non-null    int64 
 1   시장규모    352 non-null    int64 
 2   산업분류    352 non-null    object
dtypes: int64(2), object(1)
memory usage: 8.4+ KB


In [4]:
# Market capitalization data 불러오기
df = pd.read_csv('./datas/mcz_by_industry_reform.csv', encoding='euc-kr')
df_mc = pd.DataFrame(df).drop(columns='Unnamed: 0')
df_mc.head()

,년도,산업분류,시총
0,2004,"농업, 임업 및 어업",195279
1,2005,"농업, 임업 및 어업",243346
2,2006,"농업, 임업 및 어업",269164
3,2007,"농업, 임업 및 어업",735154
4,2008,"농업, 임업 및 어업",702432


In [5]:
df_mc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   년도      336 non-null    int64 
 1   산업분류    336 non-null    object
 2   시총      336 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.0+ KB


In [18]:
# Market Income data 불러오기
df = pd.read_csv('./datas/revenue_total.csv', encoding='utf-8')
df_mi = pd.DataFrame(df).rename(columns={'산업':'산업분류'})
df_mi.head()

,산업분류,년도,회사수 (개),매출액 (백만원),당기순이익 (백만원),영업이익 (백만원),경상이익 (백만원),매출액증가율 (%),매출액이익률 (%),납입자본이익률 (%),매출액경상이익률 (%),자기자본이익률 (%),총자본경상이익률 (%)
0,건설업,2004,56,37431491,1531674,2719852,2181880,13.815,4.085,29.865,5.660,11.845,6.810
1,건설업,2005,54,40785394,2029565,3096864,2885231,3.670,4.955,36.710,6.780,13.245,7.915
2,건설업,2006,57,44745972,2755247,3657870,3652228,12.410,5.850,46.305,7.950,14.790,8.855
3,건설업,2007,57,47834524,2802313,3566035,3718370,5.065,4.870,38.700,6.545,11.275,6.450
4,건설업,2008,56,53570702,3195413,3672975,4577360,16.480,4.120,37.405,6.090,9.690,5.475


In [19]:
df_mi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   산업분류          352 non-null    object 
 1   년도            352 non-null    int64  
 2   회사수 (개)       352 non-null    int64  
 3   매출액 (백만원)     352 non-null    int64  
 4   당기순이익 (백만원)   352 non-null    int64  
 5   영업이익 (백만원)    352 non-null    int64  
 6   경상이익 (백만원)    352 non-null    int64  
 7   매출액증가율 (%)    352 non-null    float64
 8   매출액이익률 (%)    352 non-null    float64
 9   납입자본이익률 (%)   352 non-null    float64
 10  매출액경상이익률 (%)  352 non-null    float64
 11  자기자본이익률 (%)   352 non-null    float64
 12  총자본경상이익률 (%)  352 non-null    float64
dtypes: float64(6), int64(6), object(1)
memory usage: 35.9+ KB


In [21]:
# MS / MC DataFrame 합치기
df_msmc = pd.merge(df_ms, df_mc, how='outer')
df_msmc = df_msmc[['산업분류', '년도', '시장규모', '시총']]
df_msmc.head()

,산업분류,년도,시장규모,시총
0,전체,1995,8.554090e+08,NaN
1,전체,2000,1.395750e+09,NaN
2,전체,2005,2.058129e+09,NaN
3,전체,2006,2.203890e+09,NaN
4,전체,2007,2.396508e+09,NaN


In [22]:
df_msmc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394 entries, 0 to 393
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   산업분류    394 non-null    object 
 1   년도      394 non-null    int64  
 2   시장규모    352 non-null    float64
 3   시총      336 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 15.4+ KB


In [23]:
df_mi.columns[2:]

Index(['회사수 (개)', '매출액 (백만원)', '당기순이익 (백만원)', '영업이익 (백만원)', '경상이익 (백만원)',
       '매출액증가율 (%)', '매출액이익률 (%)', '납입자본이익률 (%)', '매출액경상이익률 (%)',
       '자기자본이익률 (%)', '총자본경상이익률 (%)'],
      dtype='object')

In [31]:
df_msci = pd.merge(df_msmc, df_mi, how='outer')
df_msci = df_msci[['산업분류', '년도', '시장규모', '시총', '회사수 (개)', '매출액 (백만원)', '당기순이익 (백만원)', '영업이익 (백만원)', '경상이익 (백만원)',
       '매출액증가율 (%)', '매출액이익률 (%)', '납입자본이익률 (%)', '매출액경상이익률 (%)',
       '자기자본이익률 (%)', '총자본경상이익률 (%)']]
df_msci[df_msci['산업분류']=='전체'].head(10)

,산업분류,년도,시장규모,시총,회사수 (개),매출액 (백만원),당기순이익 (백만원),영업이익 (백만원),경상이익 (백만원),매출액증가율 (%),매출액이익률 (%),납입자본이익률 (%),매출액경상이익률 (%),자기자본이익률 (%),총자본경상이익률 (%)
0,전체,1995,8.554090e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,전체,2000,1.395750e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,전체,2005,2.058129e+09,NaN,1417.0,6.388000e+08,48982145.0,59244891.0,64383672.0,14.030,4.690,45.450,6.665,8.660,6.395
3,전체,2006,2.203890e+09,NaN,1509.0,6.746224e+08,42707770.0,49261879.0,53747855.0,9.150,3.460,38.115,4.885,5.995,4.560
4,전체,2007,2.396508e+09,NaN,1570.0,7.142595e+08,39091423.0,45733068.0,51549828.0,1.940,2.515,31.150,4.180,4.260,3.795
5,전체,2008,2.769841e+09,NaN,1594.0,7.645638e+08,40723551.0,50855032.0,53903447.0,11.590,0.210,19.975,1.615,-0.120,1.555
6,전체,2009,2.752651e+09,NaN,1559.0,9.503998e+08,29201182.0,55942884.0,36633205.0,14.455,2.400,29.325,3.400,4.175,2.980
7,전체,2010,3.104104e+09,NaN,1563.0,9.719882e+08,46055810.0,58399065.0,58229387.0,10.310,3.520,45.015,4.690,5.775,4.025
8,전체,2011,3.464008e+09,NaN,1548.0,1.124384e+09,72333864.0,80482156.0,87804682.0,11.400,4.595,66.830,5.885,7.495,5.125
9,전체,2012,3.541314e+09,NaN,1592.0,1.233962e+09,51907141.0,68659919.0,65283083.0,8.160,3.370,48.520,4.495,5.450,3.850


In [32]:
df_msci[df_msci['산업분류']=='제조업'].head(10)

,산업분류,년도,시장규모,시총,회사수 (개),매출액 (백만원),당기순이익 (백만원),영업이익 (백만원),경상이익 (백만원),매출액증가율 (%),매출액이익률 (%),납입자본이익률 (%),매출액경상이익률 (%),자기자본이익률 (%),총자본경상이익률 (%)
48,제조업,1995,4.054020e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,제조업,2000,6.154948e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,제조업,2005,8.805226e+08,409739490.0,995.0,419189815.0,35459443.0,41180906.0,45655220.0,12.067500,3.727778,36.552222,5.692083,6.031944,5.165833
51,제조업,2006,9.348485e+08,406539050.0,1042.0,439878648.0,28478530.0,31253741.0,35288192.0,7.941944,2.361111,31.475972,4.250417,3.748056,3.769028
52,제조업,2007,1.028049e+09,553702020.0,1049.0,430221421.0,23673026.0,26218524.0,30630616.0,3.905972,1.153333,25.368056,3.039583,2.214028,2.998750
53,제조업,2008,1.225082e+09,337905584.0,1052.0,473520394.0,25527818.0,31907466.0,34453074.0,15.209583,0.146389,12.636111,1.608750,-0.725000,1.501944
54,제조업,2009,1.186703e+09,573383927.0,1029.0,603532667.0,21828878.0,41199519.0,28361331.0,10.403611,3.162083,35.701111,4.618472,4.274861,3.749583
55,제조업,2010,1.393742e+09,739054565.0,1046.0,632855039.0,39325143.0,41878815.0,47785987.0,7.035556,4.388611,51.447917,5.845694,5.651667,4.418333
56,제조업,2011,1.608548e+09,703516911.0,1052.0,689894455.0,55826587.0,57937171.0,67074736.0,15.528472,5.134861,61.805278,6.945694,6.319444,5.272361
57,제조업,2012,1.605810e+09,789754586.0,1083.0,792591160.0,41031555.0,52818070.0,52467575.0,12.346944,3.326806,46.453611,4.797083,4.481806,3.742361


# float -> int 로 데이터 타입 변환
df_msmc[df_msmc['산업분류'] == list(df_msmc['산업분류'].unique())[0]]

for i in range(len(df_msmc['시장규모'])):
    if df_msmc['시장규모'].isna()[i] == False:
        df_msmc['시장규모'][i].astype('int')
df_msmc['시장규모']

df_msmc['시장규모'].isna()[0]

df_msmc['시총'].isna()[300]

df_msmc['시장규모'][0].astype(int)
df_msmc['시장규모'][0]

if df_msmc['시장규모'].isna()[0] == False:
    df_msmc['시장규모'][0].astype(int)
df_msmc['시장규모'][1]